In [36]:
import torch
from watermark.auto_watermark import AutoWatermark
from utils.transformers_config import TransformersConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
# Device
device = "cuda" if torch.cuda.is_available() else "cpu"


In [37]:
transformers_config = TransformersConfig(
    model=AutoModelForCausalLM.from_pretrained('Qwen/Qwen2.5-0.5B').to(device),
    tokenizer=AutoTokenizer.from_pretrained('Qwen/Qwen2.5-0.5B'),
    # vocab_size=50272,
    device=device,
    # max_new_tokens=120,
    # min_length=20,
    # do_sample=True,
    # no_repeat_ngram_size=4
)


In [38]:
 
# Load watermark algorithm
wm1 = AutoWatermark.load(
    'KGW', 
    algorithm_config='config/KGW.json',
    transformers_config=transformers_config
)
wm2 = AutoWatermark.load(
    'KGW', 
    algorithm_config='config/KGW_.json',
    transformers_config=transformers_config
)

In [39]:
# Prompt
prompt1 ="Give me a short introduction to large language model.\n"

# Generate and detect
wm1_text = wm1.generate_watermarked_text(prompt1)
wm1_detect = wm1.detect_watermark(wm1_text)
unwm1_text = wm1.generate_unwatermarked_text(prompt1)
unwm1_detect = wm1.detect_watermark(unwm1_text)
wm1_ans=wm1_text.split('\n')[-1]
unwm1_ans=unwm1_text.split('\n')[-1]
print(wm1_text)
wm1_detect

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Give me a short introduction to large language model.
A large language model, also known as a deep learning model, is a type of machine learning model that uses deep neural networks to process and analyze large amounts of data. These models are trained on large amounts of data, such as the internet or text, and can learn complex language patterns and structures from this data. Large language models are used in various applications, such as chatbots, language translation, and text generation.


{'is_watermarked': True, 'score': 5.081063304179084}

In [40]:
prompt2_wm1=\
f'Instruct:Rewrite the following text using different words:{wm1_ans}\nOutput:\n'
prompt2_unwm1=\
f'Instruct:Rewrite the following text using different words:{unwm1_ans}\nOutput:\n'

def gen_det(prompt,wm):
    wm_text = wm.generate_watermarked_text(prompt).split('\n')[-1]
    wm_detect=wm.detect_watermark(wm_text)
    unwm_text = wm.generate_unwatermarked_text(prompt).split('\n')[-1]
    unwm_detect=wm.detect_watermark(unwm_text)
    ret=dict(wm_score=wm_detect['score'],
             unwm_score=unwm_detect['score'],
             wm_text=wm_text,
             unwm_text=unwm_text,
    )
    return ret

unwm1=gen_det(prompt2_unwm1,wm2)
wm1=gen_det(prompt2_wm1,wm2)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


In [41]:

print(*unwm1.values(),sep='\n')
print(*wm1.values(),sep='\n')

0.12216944435630522
0.12216944435630522
A large language model (LLM) is a type of artificial intelligence (AI) that can generate human-like text based on a large amount of data. These models are designed to understand and generate text based on the context of the input, and they are often used for tasks such as language translation, text summarization, and chatbots.
A large language model (LLM) is a type of artificial intelligence (AI) that can generate human-like text based on a large amount of data. These models are designed to understand and generate text based on the context of the input, and they are often used for tasks such as language translation, text summarization, and chatbots.
1.2792042981336627
-0.5488212999484517
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.
A large language model, also known as a deep learning model, is a type of machine learning model that uses deep neural networks to process and analyz